Cal-Adapt Analytics Engine Data Catalog Access and Data Download
----------------------------------------------------------------

All of the climate data within the Analytics Engine is stored in a publicly accessible AWS S3 bucket. If you are familiar with programming in Python you can easily access the data using the intake package to create an xarray dataset. This xarray dataset then can be exported to NetCDF and stored physically on your computer.

In [1]:
#If running this notebook in an environment outside of the Cal-Adapt Analytics Engine Jupyter Hub make sure to install intake-esm and s3fs packages
import intake

To connect to the data catalog that stores all the relevant metadata needed to access the data issue these commands:

In [2]:
# Open catalog of available data sets using intake-esm package
cat = intake.open_esm_datastore('https://cadcat.s3.amazonaws.com/cae-collection.json')

In [3]:
# inspecting the catalog object will show the number of datasets and unique attributes
cat

,unique
activity_id,2
institution_id,3
source_id,18
experiment_id,5
member_id,15
table_id,4
variable_id,56
grid_label,3
path,5929
derived_variable_id,0


This catalog object can be converted to a Pandas dataframe to easily see the contents:

In [4]:
# Access catalog as dataframe and inspect the first few rows
cat_df = cat.df
cat_df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,path
0,LOCA2,UCSD,ACCESS-CM2,historical,r1i1p1f1,day,hursmax,d03,s3://cadcat/loca2/ucsd/access-cm2/historical/r...
1,LOCA2,UCSD,ACCESS-CM2,historical,r1i1p1f1,day,hursmin,d03,s3://cadcat/loca2/ucsd/access-cm2/historical/r...
2,LOCA2,UCSD,ACCESS-CM2,historical,r1i1p1f1,day,huss,d03,s3://cadcat/loca2/ucsd/access-cm2/historical/r...
3,LOCA2,UCSD,ACCESS-CM2,historical,r1i1p1f1,day,pr,d03,s3://cadcat/loca2/ucsd/access-cm2/historical/r...
4,LOCA2,UCSD,ACCESS-CM2,historical,r1i1p1f1,day,rsds,d03,s3://cadcat/loca2/ucsd/access-cm2/historical/r...


You can also list just the column names in the catalog by doing:

In [5]:
# Print column names
for col in cat_df:
    print(col)

activity_id
institution_id
source_id
experiment_id
member_id
table_id
variable_id
grid_label
path


To see the unique values in each column run the following code:

In [ ]:
# unique values in each column. Not all combinations of values will link to a dataset.
for col in cat_df:
    print(cat_df[col].unique())

This will give you an idea of the available query parameters that can be entered to retrieve a particular set of data. Below is a sample query against the whole catalog to refine catalog entries to those of interest.

In [ ]:
# form query dictionary
query = {
    # Downscaling method
    'activity_id': 'WRF',
    # GCM name
    #'source_id': 'ensmean',
    # time period - historical or emissions scenario
    #'experiment_id': ['historical', 'r1i1p1f1'],
    # variable
    #'variable_id': 'wind_speed_derived',
    # monthly time resolution
    'table_id': '1hr',
    # grid resolution: d01 = 45km, d02 = 9km, d03 = 3km
    #'grid_label': 'd03'
}
# subset catalog and get some metrics grouped by 'source_id'
cat_subset = cat.search(require_all_on=['source_id'], **query)
cat_subset

The zarr datasets of interest can then be loaded into memory as xarray datasets using:

In [ ]:
dsets = cat_subset.to_dataset_dict(xarray_open_kwargs={'consolidated': True},
                                   storage_options={'anon': True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


To see the dataset keys, you can list the keys.

In [ ]:
# See object keys in dsets
list(dsets)

To get down to one dataset of interest, use the key to query.

In [ ]:
# Subset to historical time period and examine data object
data = dsets['WRF.UCLA.CESM2.historical.mon.d03']
data

Finally to save a dataset to NetCDF run the last cell below.

In [ ]:
data.to_netcdf('WRF-UCLA-CESM2-historical-mon-d03.nc')